### Example: US States Data
    1. Merge and join operations come up most often when one is combining data from different sources.
    2. Here we will consider an example of some data about US states and their populations. 
    3. The data files can be found at http://github.com/jakevdp/dataUSstates/

In [1]:
import numpy as np
import pandas as pd

In [14]:
# Reading state-population.csv file
state_population = pd.read_csv('state-population.csv')
state_population.head()

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [10]:
# Reading state-areas.csv file
state_areas = pd.read_csv('state-areas.csv')
state_areas.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [13]:
# Reading state-abbrevs.csv file
state_abbrevs = pd.read_csv('state-abbrevs.csv')
state_abbrevs.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


### Problem Statement:
    Given this information, say we want to compute a relatively straightforward result:
    Rank US states and territories by their 2010 population density.
    We clearly have the data here to find this result, but we’ll have to combine the datasets to get it.

In [18]:
# Step 1 : We’ll start with a many-to-one merge that will give us the full state name within the population DataFrame
# Step 2 : We want to merge based on the state/region column of pop , and the abbreviation column of abbrevs .
# Step3 : We’ll use how='outer' to make sure no data is thrown away due to mismatched labels.
merged = pd.merge(state_population, state_abbrevs, how = 'outer', left_on = 'state/region', right_on = 'abbreviation')
merged = merged.drop('abbreviation', axis = 1)
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


In [20]:
# Let’s double-check whether there were any mismatches here, which we can do by looking for rows with nulls:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

In [22]:
merged[merged['population'].isnull()]

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN
2453,PR,under18,1993,NaN,NaN
2454,PR,under18,1992,NaN,NaN
2455,PR,total,1992,NaN,NaN
2456,PR,under18,1994,NaN,NaN
2457,PR,total,1994,NaN,NaN


In [25]:
# we see also that some of the new state entries are also null, which means that there was no corresponding entry in the abbrevs key! Let’s figure out which regions lack this match
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

In [27]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

In [30]:
# Now we can merge the result with the area data using a similar procedure
final = pd.merge(merged, state_areas, on = 'state', how = 'left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [32]:
# let’s check for nulls to see if there were any mismatches
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool